Training on Lateral sided images 

IMPORTS

In [2]:


import tarfile
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from PIL import Image
import tensorflow as tf
import cv2 as cv
from keras.models import load_model
from tensorflow.keras.applications import DenseNet121
from tqdm.notebook import tqdm
from tensorflow.keras.preprocessing.image import img_to_array

from tensorflow import concat
tf.compat.v1.enable_eager_execution()

import cv2
from skimage.transform import resize

from tensorflow.keras.layers import Input, Softmax, RNN, Dense, Embedding, LSTM,Layer,Dropout,GRU
from tensorflow.keras.models import Model
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow import repeat
from sklearn.utils import shuffle
import nltk.translate.bleu_score as bleu
import warnings
from tensorflow.keras.applications.densenet import preprocess_input

warnings.filterwarnings('ignore')


print("done importing")

done importing


File which consist image features extracted from chexNet

In [4]:

data = pd.read_pickle('final_1024.pkl')


In [5]:
data.head(5)

,filename,uid,findings,image_features,findings_total,dec_ip,dec_op
0,/home/professor/Downloads/fromgit/archive/imag...,1,The cardiac silhouette and mediastinum size ar...,"[[0.13362303376197815, 0.0, 0.0001036946268868...",<start> The cardiac silhouette and mediastinum...,<start> The cardiac silhouette and mediastinum...,The cardiac silhouette and mediastinum size ar...
1,/home/professor/Downloads/fromgit/archive/imag...,2,Borderline cardiomegaly. Midline sternotomy XX...,"[[0.11984431743621826, 0.0, 0.0006720632663927...",<start> Borderline cardiomegaly. Midline stern...,<start> Borderline cardiomegaly. Midline stern...,Borderline cardiomegaly. Midline sternotomy XX...
2,/home/professor/Downloads/fromgit/archive/imag...,3,"No displaced rib fractures, pneumothora, or pl...","[[0.12883950769901276, 0.0, 0.0003276190545875...","<start> No displaced rib fractures, pneumothor...","<start> No displaced rib fractures, pneumothor...","No displaced rib fractures, pneumothora, or pl..."
3,/home/professor/Downloads/fromgit/archive/imag...,4,There are diffuse bilateral interstitial and a...,"[[0.1524106115102768, 0.0, 0.0, 0.008435610681...",<start> There are diffuse bilateral interstiti...,<start> There are diffuse bilateral interstiti...,There are diffuse bilateral interstitial and a...
4,/home/professor/Downloads/fromgit/archive/imag...,5,The cardiomediastinal silhouette and pulmonary...,"[[0.1884971410036087, 0.0, 0.0, 0.014636723324...",<start> The cardiomediastinal silhouette and p...,<start> The cardiomediastinal silhouette and p...,The cardiomediastinal silhouette and pulmonary...


In [6]:
data.columns

Index(['filename', 'uid', 'findings', 'image_features', 'findings_total',
       'dec_ip', 'dec_op'],
      dtype='object')

0 ===> filename <br>
1 ===> uid <br>
2 ===> findings<br>
3 ===> image_features<br>
4 ===> findings_total<br>
5 ===> dec_ip<br>
6 ===> dec_op<br>

In [7]:
len(data)


6232

In [8]:
X_train, X_test = train_test_split(data.values , test_size = 0.2 )


In [10]:
X_train.shape


(4960, 7)

In [11]:
X_test.shape


(1216, 7)

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer

t1 = Tokenizer( filters='!"#$%&()*+,-/:;=?@[\\]^_`{|}~\t\n',oov_token='OOV')
t1.fit_on_texts(X_train[:,4])
vocab_size_imp = len(t1.word_index) + 1

dec_inp = t1.texts_to_sequences(X_train[:,5])

dec_inp = pad_sequences(dec_inp, maxlen=76, padding='post')

dec_inp_cv = t1.texts_to_sequences(X_test[:,5])

dec_inp_cv = pad_sequences(dec_inp_cv, maxlen=76, padding='post')

dec_op = t1.texts_to_sequences(X_train[:,6])

dec_op = pad_sequences(dec_op, maxlen=76, padding='post')

dec_op_cv = t1.texts_to_sequences(X_test[:,6])

dec_op_cv = pad_sequences(dec_op_cv, maxlen=76, padding='post')


Vocab Size

In [19]:
vocab_size_imp


2053

In [13]:
type(t1)

keras.src.preprocessing.text.Tokenizer

In [15]:
t1.word_counts

OrderedDict([('<start>', 4960),
             ('the', 8398),
             ('cardiac', 573),
             ('silhouette', 1186),
             ('is', 5731),
             ('at', 171),
             ('upper', 505),
             ('limits', 578),
             ('of', 3514),
             ('normal', 3389),
             ('for', 530),
             ('size.', 531),
             ('there', 3193),
             ('are', 5964),
             ('low', 247),
             ('lung', 922),
             ('volumes', 218),
             ('with', 962),
             ('bronchovascular', 125),
             ('crowding.', 57),
             ('no', 6568),
             ('focal', 2410),
             ('areas', 171),
             ('pulmonary', 1817),
             ('consolidation.', 560),
             ('pneumothora.', 1580),
             ('pleural', 3470),
             ('effusion.', 1431),
             ('minimal', 179),
             ('degenerative', 632),
             ('endplate', 46),
             ('changes', 743),
             ('

In [17]:
t1.word_index

{'OOV': 1,
 'the': 2,
 'no': 3,
 'are': 4,
 'is': 5,
 '<start>': 6,
 '<end>': 7,
 'and': 8,
 'of': 9,
 'pleural': 10,
 'or': 11,
 'normal': 12,
 'there': 13,
 'xxxx': 14,
 'heart': 15,
 'lungs': 16,
 'focal': 17,
 'size': 18,
 'within': 19,
 'in': 20,
 'pulmonary': 21,
 'effusion': 22,
 'normal.': 23,
 'pneumothora': 24,
 'pneumothora.': 25,
 'limits.': 26,
 'right': 27,
 'effusion.': 28,
 'mediastinal': 29,
 'clear.': 30,
 'silhouette': 31,
 'airspace': 32,
 'left': 33,
 'cardiomediastinal': 34,
 'acute': 35,
 'with': 36,
 'clear': 37,
 'consolidation': 38,
 'lung': 39,
 'stable': 40,
 'a': 41,
 'structures': 42,
 'mediastinum': 43,
 'changes': 44,
 'thoracic': 45,
 'unremarkable.': 46,
 'bony': 47,
 'mild': 48,
 'degenerative': 49,
 'spine.': 50,
 'without': 51,
 'limits': 52,
 'contours': 53,
 'cardiac': 54,
 'disease.': 55,
 'calcified': 56,
 'appear': 57,
 'consolidation.': 58,
 'large': 59,
 'size.': 60,
 'for': 61,
 'osseous': 62,
 'to': 63,
 'upper': 64,
 'vascularity': 65,
 'x

In [18]:
with open('t2.pickle', 'wb') as handle:
    pickle.dump(t1, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [20]:
t2 = pd.read_pickle('/home/professor/Downloads/fromgit/Deadline/t2.pickle')


In [21]:


imp1 = {}
imp2 = {}
for key,value in t2.word_index.items():
  imp1[value] = key
  imp2[key] = value


In [22]:
imp1

{1: 'OOV',
 2: 'the',
 3: 'no',
 4: 'are',
 5: 'is',
 6: '<start>',
 7: '<end>',
 8: 'and',
 9: 'of',
 10: 'pleural',
 11: 'or',
 12: 'normal',
 13: 'there',
 14: 'xxxx',
 15: 'heart',
 16: 'lungs',
 17: 'focal',
 18: 'size',
 19: 'within',
 20: 'in',
 21: 'pulmonary',
 22: 'effusion',
 23: 'normal.',
 24: 'pneumothora',
 25: 'pneumothora.',
 26: 'limits.',
 27: 'right',
 28: 'effusion.',
 29: 'mediastinal',
 30: 'clear.',
 31: 'silhouette',
 32: 'airspace',
 33: 'left',
 34: 'cardiomediastinal',
 35: 'acute',
 36: 'with',
 37: 'clear',
 38: 'consolidation',
 39: 'lung',
 40: 'stable',
 41: 'a',
 42: 'structures',
 43: 'mediastinum',
 44: 'changes',
 45: 'thoracic',
 46: 'unremarkable.',
 47: 'bony',
 48: 'mild',
 49: 'degenerative',
 50: 'spine.',
 51: 'without',
 52: 'limits',
 53: 'contours',
 54: 'cardiac',
 55: 'disease.',
 56: 'calcified',
 57: 'appear',
 58: 'consolidation.',
 59: 'large',
 60: 'size.',
 61: 'for',
 62: 'osseous',
 63: 'to',
 64: 'upper',
 65: 'vascularity',
 66

In [23]:
imp2

{'OOV': 1,
 'the': 2,
 'no': 3,
 'are': 4,
 'is': 5,
 '<start>': 6,
 '<end>': 7,
 'and': 8,
 'of': 9,
 'pleural': 10,
 'or': 11,
 'normal': 12,
 'there': 13,
 'xxxx': 14,
 'heart': 15,
 'lungs': 16,
 'focal': 17,
 'size': 18,
 'within': 19,
 'in': 20,
 'pulmonary': 21,
 'effusion': 22,
 'normal.': 23,
 'pneumothora': 24,
 'pneumothora.': 25,
 'limits.': 26,
 'right': 27,
 'effusion.': 28,
 'mediastinal': 29,
 'clear.': 30,
 'silhouette': 31,
 'airspace': 32,
 'left': 33,
 'cardiomediastinal': 34,
 'acute': 35,
 'with': 36,
 'clear': 37,
 'consolidation': 38,
 'lung': 39,
 'stable': 40,
 'a': 41,
 'structures': 42,
 'mediastinum': 43,
 'changes': 44,
 'thoracic': 45,
 'unremarkable.': 46,
 'bony': 47,
 'mild': 48,
 'degenerative': 49,
 'spine.': 50,
 'without': 51,
 'limits': 52,
 'contours': 53,
 'cardiac': 54,
 'disease.': 55,
 'calcified': 56,
 'appear': 57,
 'consolidation.': 58,
 'large': 59,
 'size.': 60,
 'for': 61,
 'osseous': 62,
 'to': 63,
 'upper': 64,
 'vascularity': 65,
 'x



##Model Design Of LLM##

In [163]:
@tf.keras.utils.register_keras_serializable()

class Encoder(tf.keras.Model):
    '''
    Encoder model -- That takes a input sequence and returns output sequence
    '''

    def __init__(self,units):
        super().__init__()
        self.units = units
        self.dense = Dense(self.units,name = 'Enc_dense')


    def call(self,img):
      '''
          This function takes a sequence input and the initial states of the encoder.
          Pass the input_sequence input to the Embedding layer, Pass the embedding layer ouput to encoder_lstm
          returns -- All encoder_outputs, last time steps hidden and cell state
      '''
      #enc_out = self.maxpool(tf.expand_dims(img,axis = 2))
      enc_out = self.dense(img)
      return enc_out


    def initialize_states(self,batch_size):
      '''
      Given a batch size it will return intial hidden state and intial cell state.
      If batch size is 32- Hidden state is zeros of size [32,lstm_units], cell state zeros is of size [32,lstm_units]
      '''
      self.batch_size  = batch_size

      self.enc_h =tf.zeros((self.batch_size, self.units))

        
      #self.enc_c = tf.zeros((self.batch_size, self.lstm_size))
      return self.enc_h


In [164]:
@tf.keras.utils.register_keras_serializable()

class Attention(tf.keras.layers.Layer):
  '''
    Class the calculates score based on the scoring_function using Bahdanu attention mechanism.
  '''
  def __init__(self,att_units):
    # Please go through the reference notebook and research paper to complete the scoring functions
    super().__init__()

    self.att_units = att_units

    self.w1 =  tf.keras.layers.Dense( self.att_units , name = 'w1')
    self.w2 =  tf.keras.layers.Dense( self.att_units,name = 'w2')
    self.v =  tf.keras.layers.Dense(1,name = 'v')
      

  def call(self,decoder_hidden_state,encoder_output):
    '''
      Attention mechanism takes two inputs current step -- decoder_hidden_state and all the encoder_outputs.
      * Based on the scoring function we will find the score or similarity between decoder_hidden_state and encoder_output.
        Multiply the score function with your encoder_outputs to get the context vector.
        Function returns context vector and attention weights(softmax - scores)
    '''
    print("Attention ===============================")
    self.decoder_hidden_state = decoder_hidden_state
    '''
       decoder_hidden_state.shape =  (32, 256)
    '''
    self.encoder_output = encoder_output
    '''
        encoder_output.shape =  (32, 256)
    '''
    print("Atten layer")
    print("Dec shape ",decoder_hidden_state.shape, "Enc shape", encoder_output.shape )
      



    self.decoder_hidden_state = tf.expand_dims(self.decoder_hidden_state,axis = 1)
    score = self.v(tf.nn.tanh(
              self.w1(self.decoder_hidden_state) + self.w2(self.encoder_output)))
    attention_weights = tf.nn.softmax(score, axis=1)
    '''
        attention_weights.shape = (32, 32, 1)
    '''
    context_vector = attention_weights * self.encoder_output
    
    context_vector = tf.reduce_sum(context_vector, axis=1)
    '''
        context_vector.shape = (32, 256)
    '''
    print("attention_weights.shape = ", attention_weights.shape)
    print("context_vector.shape = ", context_vector.shape)
    print("encoder_output.shape = ", encoder_output.shape)
    print("decoder_hidden_state.shape = ", decoder_hidden_state.shape)
    
    print("Attention ===============================END")
    return context_vector,attention_weights


In [165]:
@tf.keras.utils.register_keras_serializable()

class OneStepDecoder(tf.keras.Model):
  def __init__(self,tar_vocab_size, embedding_dim, input_length, dec_units  ,att_units):

      # Initialize decoder embedding layer, LSTM and any other objects needed
      super().__init__()
      self.tar_vocab_size = tar_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.dec_emb = Embedding(tar_vocab_size,embedding_dim,trainable = True , name = 'dec_embb')
      self.dec_lstm = GRU(self.dec_units, return_state=True, return_sequences=True, name="Decoder_LSTM")
      self.dense   = Dense(self.tar_vocab_size, name = 'one_dec')
      self.attention=Attention( self.att_units)
      self.d1 = Dropout(0.3,name = 'd1')
      self.d2 = Dropout(0.3,name = 'd2')
      self.d3 = Dropout(0.3,name = 'd3')

            

  @tf.function
  def call(self,input_to_decoder, encoder_output, state_h):
    print("ONestepDecoder ================")
    '''
        One step decoder mechanisim step by step:
      A. Pass the input_to_decoder to the embedding layer and then get the output(batch_size,1,embedding_dim)
      B. Using the encoder_output and decoder hidden state, compute the context vector.
      C. Concat the context vector with the step A output
      D. Pass the Step-C output to LSTM/GRU and get the decoder output and states(hidden and cell state)
      E. Pass the decoder output to dense layer(vocab size) and store the result into output.
      F. Return the states from step D, output from Step E, attention weights from Step -B
    '''
    self.input_to_decoder = input_to_decoder
    '''
        input_to_decoder.shape = (32, None)
    '''
    self.encoder_output = encoder_output
    '''
        encoder_output.shape = (32, 256)
    '''
    self.state_h = state_h
    '''
        state_h.shape = (32,256)
    '''
    #A
    target_embedd           = self.dec_emb (self.input_to_decoder)     #(batch_size,1,embedingdim)
    #B
    target_embedd = self.d1(target_embedd)
    '''
        target_embedd.shape = (32, None, 300)
    '''

    context_vector,attention_weights=self.attention(self.state_h,self.encoder_output) #context vector shape = (batch_size,att_units)
    '''
        context_vector.shape = (32, 256)
        attention_weights.shape = (32, 32, 1)
        
    '''
    #C
    concated = tf.concat([  tf.expand_dims(context_vector, 1),target_embedd], -1)
    concated = self.d2(concated)
    '''
        concated.shape = (32, 1, 556)
    '''

    #D
    lstm_output, hs      = self.dec_lstm(concated, initial_state=self.state_h)
    '''
        lstm_output.shape = (32, 1, 256)
        hs.shape = (32, 256)
    '''
    


    lstm_output = tf.reshape(lstm_output, (-1, lstm_output.shape[2]))
    lstm_output = self.d3(lstm_output)

    '''
        lstm_output.shape = (32, 256)
    '''
    #E
    op = self.dense(lstm_output)
    '''
        op.shape = (32, 2053)
    '''
    #op = tf.squeeze(op,[1])
    
    print("input_to_decoder :- ", input_to_decoder.shape , "state_h :- ", state_h.shape)
    
    print("encoder_output ", state_h.shape, "Target eMBEDING - 2",target_embedd.shape )
    print("Concat :- ", concated.shape)
    print("Lstm output",lstm_output.shape, "OP", op.shape )
    print("HS:- ", hs.shape)
    print("context_vector.shape", context_vector.shape,"attention_weights.shape", attention_weights.shape )
    
    print("ONestepDecoder ================END")

    # print("dec_lstm:- ", dec_lstm)
    return op,hs,attention_weights,context_vector


In [166]:
@tf.keras.utils.register_keras_serializable()

class Decoder(tf.keras.Model):
    '''
        out_vocab_size = 2053
        embedding_dim = 300
        input_length = 76
        dec_units = 256
        att_units = 64
    '''
    def __init__(self,out_vocab_size, embedding_dim, input_length, dec_units ,att_units):
      #Intialize necessary variables and create an object from the class onestepdecoder
      super().__init__()
      self.out_vocab_size = out_vocab_size
      self.embedding_dim = embedding_dim
      self.input_length = input_length
      self.dec_units = dec_units
      self.att_units = att_units
      self.onestep = OneStepDecoder(self.out_vocab_size,self.embedding_dim ,self.input_length,self.dec_units,self.att_units)

        
    @tf.function
    def call(self, input_to_decoder,encoder_output,decoder_hidden_state):


        #Initialize an empty Tensor array, that will store the outputs at each and every time step
        #Create a tensor array as shown in the reference notebook

        #Iterate till the length of the decoder input
            # Call onestepdecoder for each token in decoder_input
            # Store the output in tensorarray
        # Return the tensor array
        print("DECODER =======================")
        all_outputs = tf.TensorArray(tf.float32,size =input_to_decoder.shape[1],name = 'output_arrays' )
        '''
            all_outputs.shape = (32, 76, 2053)
        '''
        self.input_to_decoder = input_to_decoder
        '''
            input_to_decoder.shape = (32, 76)
        '''
        self.encoder_output = encoder_output
        '''
            encoder_output = (32, 256)
        '''
        
        self.decoder_hidden_state = decoder_hidden_state
        '''
            decoder_hidden_state.shape = (32, 256)
        '''

        for timestep in tf.range(input_to_decoder.shape[1]):
          op,hs,attention_weights,context_vector = self.onestep(self.input_to_decoder[:,timestep:timestep+1], self.encoder_output, self.decoder_hidden_state)
          self.decoder_hidden_state = hs
          all_outputs = all_outputs.write(timestep,op)
        all_outputs = tf.transpose(all_outputs.stack(),[1,0,2])
        '''
            all_outputs.shape = (32, 76, 2053)
        '''

        print("input_to_decoder" , input_to_decoder.shape)
        print("decoder_hidden_state", decoder_hidden_state.shape)
        print("all_outputs", all_outputs.shape)
        print("encoder_output = ", encoder_output.shape)
        print("DECODER =======================END")
        return all_outputs


In [187]:
@tf.keras.utils.register_keras_serializable()
class encoder_decoder(tf.keras.Model):
  #def __init__(self,#params):
    #Intialize objects from encoder decoder
  
  '''out_vocab_size = 2053
      embedding_size_d = 300
      input_length_d = 76
      lstm_size_d = 256
      att_units = 64
      batch_size = 32
      units = 256
  '''  
  def __init__(self,out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size,units):

        #Create encoder object
        #Create decoder object
        #Intialize Dense layer(out_vocab_size) with activation='softmax'

        
        super().__init__()

        self.units = units
        self.out_vocab_size = out_vocab_size
        self.embedding_size_d = embedding_size_d
        self.lstm_size_d = lstm_size_d
        self.input_length_d = input_length_d
        self.batch_size = batch_size
        self.att_units = att_units

        self.encoder = Encoder(self.units)

        self.decoder = Decoder(out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units )
        self.dense   = Dense(self.out_vocab_size,name = 'enc_dec_dense')



  def call(self,data):
    #Intialize encoder states, Pass the encoder_sequence to the embedding layer
    # Decoder initial states are encoder final states, Initialize it accordingly
    # Pass the decoder sequence,encoder_output,decoder states to Decoder
    # return the decoder output
    print("encoder_decoder ===================")
    self.inputs,self.outputs = data[0], data[1]
    '''
        Data[0] value  =  (32,1024)
        Data[1] value  =  (32,76)
        
    '''
    # print("="*20, "ENCODER", "="*20)
    self.encoder_h= self.encoder.initialize_states(self.batch_size)
    self.encoder_output = self.encoder(self.inputs)
    '''
        encoder_output.shape = (32,256)
        encoder_h.shape = (32,256)
    '''
    print("-"*27)
    # print("ENCODER ==> OUTPUT SHAPE",self.encoder_output.shape)
    # print("ENCODER ==> HIDDEN STATE SHAPE",self.encoder_h.shape)
    # print("="*20, "DECODER", "="*20)
    output= self.decoder(self.outputs,self.encoder_output,self.encoder_h)
    '''
        output.shape = (32, 76, 2053)
        
    '''
    # print("FINAL OUTPUT SHAPE",output.shape)
    print("data[0], data[1]  = ", data[0].shape, data[1].shape)
    print("output.shape = ", output.shape)
    print("encoder_decoder =================== END")
    return output


In [188]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)
  #out_vocab_size , embedding_size_d, input_length_d,lstm_size_d,att_units,batch_size)


In [189]:
!mkdir checkpoint


mkdir: cannot create directory ‘checkpoint’: File exists


In [190]:
import os
import tensorflow as tf

# Define current working directory
cwd = os.getcwd()

# Define checkpoint directory and file path
checkpoint_dir = os.path.join(cwd, 'checkpoint')
checkpoint_filepath = os.path.join(checkpoint_dir, 'model_checkpoint.keras')

# Ensure the checkpoint directory exists
os.makedirs(checkpoint_dir, exist_ok=True)

# Create the ModelCheckpoint callback
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_loss',
    verbose=1,
    mode='min',
    save_best_only=True
)

# Print checkpoint filepath to verify
print("Checkpoint filepath:", checkpoint_filepath)



Checkpoint filepath: /home/professor/Downloads/fromgit/Deadline/checkpoint/model_checkpoint.keras


In [191]:
tf.keras.backend.clear_session()


In [192]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8,mode = 'min',verbose = 1,
                              patience=2, min_lr=0.0001)


In [193]:
model_1 = encoder_decoder(vocab_size_imp,300,76,256,64,32,256)
optimizer = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimizer,loss=loss_function, metrics = ["accuracy"])


In [194]:
train_inp = np.vstack(X_train[:,3]).astype(float)  # Use np.float64 for double-precision floats
test_inp = np.vstack(X_test[:,3]).astype(float)  # Use np.float32 for single-precision floats (if memory efficiency is a concern)


In [195]:
# Print the shapes of your data
print("train_inp shape:", train_inp.shape)
print("dec_inp shape:", dec_inp.shape)
print("dec_op shape:", dec_op.shape)
print("test_inp shape:", test_inp.shape)
print("dec_inp_cv shape:", dec_inp_cv.shape)
print("dec_op_cv shape:", dec_op_cv.shape)


train_inp shape: (4960, 1024)
dec_inp shape: (4960, 76)
dec_op shape: (4960, 76)
test_inp shape: (1216, 1024)
dec_inp_cv shape: (1216, 76)
dec_op_cv shape: (1216, 76)


In [196]:
model_1.fit([train_inp,dec_inp ],dec_op ,validation_data= ([test_inp, dec_inp_cv],dec_op_cv),batch_size= 32,epochs  = 1,callbacks=[reduce_lr,model_checkpoint_callback] , shuffle=True)


encoder_decoder ===================
---------------------------
DECODER =======================
ONestepDecoder ================
Attention ===============================
Atten layer
Dec shape  (32, 256) Enc shape (32, 256)
attention_weights.shape =  (32, 32, 1)
context_vector.shape =  (32, 256)
encoder_output.shape =  (32, 256)
decoder_hidden_state.shape =  (32, 256)
Attention ===============================END
input_to_decoder :-  (32, None) state_h :-  (32, 256)
encoder_output  (32, 256) Target eMBEDING - 2 (32, None, 300)
Concat :-  (32, 1, 556)
Lstm output (32, 256) OP (32, 2053)
HS:-  (32, 256)
context_vector.shape (32, 256) attention_weights.shape (32, 32, 1)
ONestepDecoder ================END
ONestepDecoder ================
Attention ===============================
Atten layer
Dec shape  (32, 256) Enc shape (32, 256)
attention_weights.shape =  (32, 32, 1)
context_vector.shape =  (32, 256)
encoder_output.shape =  (32, 256)
decoder_hidden_state.shape =  (32, 256)
Attention =======